#### Quick Start: How to Use the RAG Framework in Your Application

This example demonstrates the basic usage of the framework via a YAML configuration file and a Dependency Injection (DI) container.

---

This framework provides the infrastructure for building Retrieval-Augmented Generation (RAG) systems in Python.  
It is designed according to the principles of Clean Architecture and uses YAML-based configuration and a DI container  
to connect components such as retrievers, LLMs, data loaders, and more.


#### Framework Architecture

The framework is designed in accordance with the principles of Clean Architecture:

- **Ports** — interfaces that define what a component must implement (e.g., `LLMPort`, `RetrieverPort`).
- **Adapters** — concrete implementations of those interfaces, such as `HuggingFaceInferenceAdapter`.
- **DIContainer** — assembles adapters based on the configuration without violating isolation or scalability.

This allows you to easily swap out adapters (e.g., replace HuggingFace with OpenAI or your own implementation) without touching the business logic.

#### Glossary of Key Framework Components

- **IngestionService** — responsible for data preparation: loading, chunking, and indexing in the retriever.
- **AnswerService** — handles the main answering pipeline: extracting relevant documents and generating a response using the LLM.
- **Retriever** — component responsible for searching documents. Its interface is defined in `RetrieverPort`.
- **LLM** — large language model. The interface for response generation is defined in `LLMPort`.
- **DIContainer** — a dependency injection container that builds and manages component lifecycles.
- **ADAPTER_REGISTRY** — a global registry of interface implementations (adapters), available for configuration-based injection.

The easiest way to get started with the framework is via a YAML configuration file and the dependency injection container.

All you need to do is:

- load a configured YAML file,
- create an instance of the `DIContainer` and call `build_app_service()` to initialize the components,
- use the `generate_answer` method to generate a response based on retrieved relevant documents.


Проще всего начать использование фреймворка с YAML-конфигурации и контейнера зависимостей.

Все, что от вас требуется:
- загрузить настроенный файл конфигурации,
- создать экземпляр конструктора зависимостей `DIContainer` и вызвать метод построения этих зависимостей на основе файла конфигурации `build_app_service`,
- воспользоваться методом генерации ответа на основе найденных релевантных документов `generate_answer`.

In [2]:
from ragbee_fw import load_config

In [3]:
app_config_path = "/workspace/src/ragbee_fw/config/app_config.yml"
app_config = load_config(app_config_path)
app_config.llm.token = "your_token"

In [4]:
from ragbee_fw import DIContainer


container = DIContainer(app_config)
app = container.build_app_service()

#### Result: Working with `AnswerService`

As a result, we obtain an instance of the `AnswerService` class, which is designed for the main pipeline:

- retrieving text fragments relevant to the user query,
- generating a response using the LLM.


In [14]:
type(app)

ragbee_fw.core.services.answer_service.AnswerService

You can now simply pass a question to the `generate_answer` method of the created `AnswerService` instance and get a response from the LLM.

In [ ]:
response = app.generate_answer(query="какие были Испанские завоевания в Америке?")

In [11]:
response

'Испанские завоевания в Америке включали в себя несколько важных событий и кампаний, которые привели к колонизации значительной части американского континента. Вот ключевые моменты:\n\n1. **Экспедиция Кортеса в Мексику (1518 г.)**: Испанский губернатор Кубы отправил экспедицию под руководством Фернандо Кортеса в Мексику. Кортес высадился с 600 человек и, чтобы исключить возможность отступления, сжег все корабли.\n\n2. **Завоевание государства ацтеков (1521 г.)**: Кортес двинулся вглубь страны и достиг столицы ацтеков, Мехико. Первоначально ацтеки приняли испанцев за богов, но мирные отношения быстро закончились, и испанцы учинили погром. В 1521 году Кортес окончательно захватил и разграбил Мехико, а страну объявили собственностью испанского короля.\n\n3. **Завоевание империи инков (1532–1534 гг.)**: Франциско Писарро вторгся в империю инков в Южной Америке, захватил в плен их правителя, и к 1534 году государство инков было уничтожено.\n\n4. **Завоевание Венесуэлы (1525–1535 гг.)**: Исп

---

> **Note:** The DI container keeps all created objects in an internal dictionary called `_cache`.  
> This may be useful when integrating custom modules or accessing shared dependencies manually.

In [24]:
contain_obj = container._cache
display(contain_obj.keys())

dict_keys(['data_loader', 'text_chunker', 'retriever', 'retriever_with_index', 'llm'])

In [25]:
retriever = contain_obj.get("retriever")
display(retriever)

#### Low-Level Control Over Dependencies and Custom Module Creation

You can bypass the DI container and manually construct all dependencies.  
This provides maximum flexibility and control when integrating custom modules.

However, this approach requires a deeper understanding of the framework's architecture, including the relationships between ports and adapters in the spirit of hexagonal architecture.


In [5]:
from ragbee_fw.infrastructure.data_loader.file_loader import FileSystemLoader
from ragbee_fw.infrastructure.text_splitter.recursive_text_splitter import RecursiveTextSplitter
from ragbee_fw.infrastructure.retriever.bm25_client import BM25Client 
from ragbee_fw.infrastructure.llm_clients.huggingface_client import HuggingFaceInferenceAdapter

from ragbee_fw import IngestionService
from ragbee_fw import AnswerService

In [ ]:
loader = FileSystemLoader()
chanker = RecursiveTextSplitter()
retriever = BM25Client()
llm = HuggingFaceInferenceAdapter(model_name="meta-llama/Llama-4-Scout-17B-16E-Instruct",
                                    provider="cerebras",
                                    token="your_token",)


At this point, you can build and inject any custom module into the pipeline that aligns with one of the following responsibilities:

- data loading, chunking, and indexing — for `IngestionService`,  
- document retrieval and answer generation — for `AnswerService`.

All you need is to follow the port-based connection rules described in `ragbee_fw.core.ports`.


In [ ]:
ingestion = IngestionService(
    loader=loader,
    chunker=chanker,
    retriever=retriever
)
retriever = ingestion.build_index("/workspace/documents")

In [ ]:
responsible = AnswerService(
    retriever=retriever,
    llm=llm
)
response = responsible.generate_answer(
    query="какие были Испанские завоевания в Америке?"
)

In [ ]:
response

'Испанские завоевания в Америке включали в себя несколько ключевых событий и завоеваний, которые существенно повлияли на историю региона. Вот основные моменты:\n\n1. **Экспедиция Кортеса в Мексику (1518 г.)**: Испанский губернатор острова Кубы отправил экспедицию под предводительством Фернандо Кортеса в Мексику. Кортес сжег все корабли после высадки, чтобы исключить возможность возвращения, и двинулся вглубь страны к государству ацтеков.\n\n2. **Захват Мехико (1521 г.)**: Испанцы беспрепятственно вошли в столицу ацтеков, Мехико, где их приняли за богов. Однако мирные отношения быстро закончились, и испанцы учинили погром. В 1521 году Кортес окончательно захватил и разграбил Мехико, объявив страну собственностью испанского короля.\n\n3. **Завоевание империи инков (1532 г.)**: Франциско Писарро вторгся в империю инков в Южной Америке и захватил в плен их правителя. К 1534 году государство инков было уничтожено.\n\n4. **Завоевание Венесуэлы (1525–1535 гг.)**: Испанцы окончательно завоевал

#### Creating a Custom Module and Registering it in the Dependency Container

The easiest way to integrate your own module is by registering it in the DI container through `ADAPTER_REGISTRY`.  
This lowers the entry barrier and simplifies integration.

Steps:

- Implement your custom class that adheres to the port interface (see `ragbee_fw.core.ports`).  
  You’ll also find abstract base classes there to guide and validate your implementation.
- Register your module in the `ADAPTER_REGISTRY`.
- Define your module in the `config.yml` file.
- Proceed with the standard generation process using the DI container.


In [1]:
from ragbee_fw.core.ports.llm_port import LLMPort, BaseLLM
from ragbee_fw import DIContainer, ADAPTER_REGISTRY


In [2]:
display(ADAPTER_REGISTRY)

{'data_loader': {'file_loader': ragbee_fw.infrastructure.data_loader.file_loader.FileSystemLoader},
 'text_chunker': {'recursive_splitter': ragbee_fw.infrastructure.text_splitter.recursive_text_splitter.RecursiveTextSplitter},
 'retriever': {'bm25': ragbee_fw.infrastructure.retriever.bm25_client.BM25Client},
 'llm': {'HF': ragbee_fw.infrastructure.llm_clients.huggingface_client.HuggingFaceInferenceAdapter}}

In [ ]:
class MyDummyResponce(LLMPort):
    def generate(self, prompt: str):
        # custom logic goes here
        return f"I got {len(prompt)} chars in out prompt: {prompt}"


In [4]:
DIContainer.register_adapter(component="llm", 
                           adapter_type="dummy",
                           cls=MyDummyResponce)

In [5]:
display(ADAPTER_REGISTRY)

{'data_loader': {'file_loader': ragbee_fw.infrastructure.data_loader.file_loader.FileSystemLoader},
 'text_chunker': {'recursive_splitter': ragbee_fw.infrastructure.text_splitter.recursive_text_splitter.RecursiveTextSplitter},
 'retriever': {'bm25': ragbee_fw.infrastructure.retriever.bm25_client.BM25Client},
 'llm': {'HF': ragbee_fw.infrastructure.llm_clients.huggingface_client.HuggingFaceInferenceAdapter,
  'dummy': __main__.MyDummyResponce}}

Once registered, your module becomes available to the DI container and can replace the default implementation in the pipeline.

The next step is to define it in the `config.yml` file:

```yaml
# config_app.yml
llm:
  type: dummy
  model_name: any_name
  token: ""
  provider: ""
  prompt: ""
  max_new_tokens: 0
```

In this example, however, we will directly modify the loaded Pydantic config object instead of editing the YAML file.

In [6]:
from ragbee_fw import load_config

app_config_path = "/workspace/src/ragbee_fw/config/app_config.yml"
app_config = load_config(app_config_path)

In [7]:
from ragbee_fw.core.models.app_config import LLM


app_config.llm = LLM(**{
    "type": "dummy",
    "model_name": "any_name",
    "token": "",
    "provider": "",
    "prompt": "",
    "max_new_tokens": 0,
})

> 💡 **Tip:** The `load_config(...)` function returns a Pydantic model instance.  
> This means you can interact with it just like any other Python object:
>
> - update fields directly (`app_config.llm = ...`),
> - validate values,
> - export using `.dict()` or `.json()`.
>
To save the configuration back to YAML:

```python
from pathlib import Path
import yaml

with Path("new_config.yml").open("w", encoding="utf-8") as f:
    yaml.safe_dump(app_config.dict(), f, allow_unicode=True)
```

In [8]:
display(app_config.llm)

LLM(type='dummy', model_name='any_name', token='', provider='', base_url=None, prompt='', max_new_tokens=0, return_full_response=False, params=None)

Now you have an application configuration object that includes your custom LLM module (e.g., `MyDummyResponse`).  
From here, you can proceed with the standard framework usage.

In [9]:
container = DIContainer(app_config)
app = container.build_app_service()

In [10]:
response = app.generate_answer(query="какие были Испанские завоевания в Америке?")

In [11]:
response

'I got 4150 chars in out prompt: Based on the following fragments:\n\n[1]  прибыль.   Испанские завоевания в Америке.\n   В 1518 г. испанский губернатор острова Кубы послал экспедицию из 600 человек во главе с Фернандо Кортесом в только что открытую Мексику. Кортес сжег после высадки все корабли, чтобы нельзя было вернуться домой, и двинулся в глубь страны к государству ацтеков. Испанцы беспрепятственно проникли в столицу ацтеков Мехико. Ацтеки приняли белокожих испанцев за богов. Но мирные отношения быстро закончились. Испанцы учинили страшный погром. В 1521 г. Кортес окончательно захватил и разграбил Мехико. Страна была объявлена собственностью испанского короля.\n   В 1532 г. Франциска Писарро вторгся в империю инков в Южной Америке и захватил в плен их правителя. К 1534 г. государство инков было уничтожено. В 1525–1535 гг. испанцы окончательно завоевали Венесуэлу. Здесь было найдено золото и возникла легенда о волшебной стране Эльдорадо, в которой много золота. Поиски Эльдорадо был